# Part 3: LGBM Model Development

In [29]:
import pandas as pd
ml_df = pd.read_csv('data/racedata_2017-2022_ETL.csv', index_col = 0)

In [30]:
ml_df = ml_df.dropna()

In [31]:
import os
import pickle
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error

from sklearn.feature_selection import RFE

working_directory = 'D:/machine_learning/DFS/nascar/'
data_dir = 'data/'

# needs hyperparameters

dcols = [
    'Pos',
     'St',
     '#',
     'Driver',
     'Laps',
     'Led',
     'Pts',
     'PPts',
     'Track',
     'race_ID',
     'FP',
     'place_diff',
     'Date',
     'Date_dtobj',
     'Year',
     'DNF',
     'Win',
     'T5',
     'T10',
     'T20',
     'track_type'
]

X = ml_df.drop(dcols, axis = 1)
Y = ml_df['FP']

from sklearn.model_selection import train_test_split

#Create Training and Testing DataSets
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.25, random_state=42)

X_train.reset_index(inplace = True, drop=True)
X_test.reset_index(inplace = True, drop=True)
Y_train.reset_index(inplace = True, drop=True)
Y_test.reset_index(inplace = True, drop=True)

print('Training set size:', len(X_train))
print('Testing set size:', len(X_test))

pred_df = pd.concat([X_test, Y_test], axis = 1)

print('\nNum Possible Features:',len(X_train.columns.tolist()))

Training set size: 6234
Testing set size: 2078

Num Possible Features: 67


In [32]:
# get statuses for comparison
train_status = X_train['Status']
test_status = X_test['Status']

X_train.drop(['Status'], axis = 1, inplace = True)
X_test.drop(['Status'], axis = 1, inplace = True)

pred_df['Status'] = test_status

C:\Users\Greg\anaconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [33]:
# standardize data
from sklearn.preprocessing import StandardScaler, MinMaxScaler, Normalizer
from sklearn.compose import ColumnTransformer

# get numerical features from the features dataframe and store numerical columns in a variable
numerical_features = X_train.select_dtypes(include=['float64', 'int64'])
numerical_columns = numerical_features.columns
 
ct = ColumnTransformer([("only numeric", StandardScaler(), numerical_columns)], remainder='passthrough')

# fit columntransformer to the training data and transform it
X_train_scaled = ct.fit_transform(X_train)

# fit columntransformer to the testing data and transform it
X_test_scaled = ct.transform(X_test)

# convert to pd dataframes
X_train_scaled = pd.DataFrame(X_train_scaled, columns = X_train.columns)
X_test_scaled = pd.DataFrame(X_train_scaled, columns = X_test.columns)

In [35]:
X_train_scaled

,LL_yr_cumsum,LC_yr_cumsum,DNF_yr_cumsum,Win_yr_cumsum,T5_yr_cumsum,T10_yr_cumsum,T20_yr_cumsum,FinPos_mean_L3,StrtPos_mean_L3,PlaceDiff_mean_L3,...,Pts_mean_L3_tt,PPts_sum_L3_tt,PPts_mean_L3_tt,FP_sum_L3_tt,FP_mean_L3_tt,DNF_sum_L3_tt,Win_sum_L3_tt,T5_sum_L3_tt,T10_sum_L3_tt,T20_sum_L3_tt
0,-0.488737,-0.305571,-0.687274,-0.421026,-0.655717,-0.485303,-0.406760,0.443735,0.094085,-0.448399,...,-0.577473,-0.293484,-0.292616,-0.422855,-0.531861,1.003053,-0.252433,-0.539774,-0.774935,-0.387907
1,-0.477410,0.270799,0.310506,-0.421026,-0.655717,-0.306698,0.201302,0.747097,-0.307039,-1.457496,...,0.109553,-0.293484,-0.292616,-0.171780,-0.272870,-0.631299,-0.252433,-0.539774,0.279857,0.487306
2,-0.462308,0.636707,0.310506,-0.421026,-0.655717,-0.663907,0.201302,-0.087148,0.428356,0.762516,...,-0.094010,-0.293484,-0.292616,0.120590,0.028719,-0.631299,-0.252433,-0.539774,-0.774935,0.487306
3,-0.492512,-1.500034,0.310506,-0.421026,-0.655717,-0.842511,-1.136435,1.846785,1.531449,-0.145671,...,-1.417172,-0.293484,-0.292616,-1.703005,-1.852378,2.637405,-0.252433,-0.539774,-0.774935,-1.263121
4,-0.484961,-0.644894,-1.186164,-0.421026,-0.379481,-0.485303,-0.649985,-0.580112,0.394929,1.367974,...,0.949252,-0.293484,-0.292616,-0.558303,2.633971,-0.631299,-0.252433,-0.539774,0.279857,-0.387907
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6229,-0.492512,-1.384833,-0.687274,-0.421026,-0.655717,-0.663907,-0.893210,0.822938,0.929762,0.308423,...,0.033217,-0.293484,-0.292616,0.000008,-0.095665,-0.631299,-0.252433,-0.539774,-0.774935,-0.387907
6230,-0.488737,-0.980895,-0.687274,-0.421026,-0.655717,-0.842511,-0.771598,0.102453,0.628918,0.812971,...,-1.417172,-0.293484,-0.292616,0.175100,0.084948,1.003053,-0.252433,-0.539774,-0.774935,0.487306
6231,-0.488737,-1.538434,-1.186164,-0.421026,-0.655717,-0.842511,-1.014823,1.770944,1.498021,-0.095216,...,-1.035491,-0.293484,-0.292616,-0.787904,-0.908422,1.003053,-0.252433,-0.539774,-0.774935,-1.263121
6232,-0.303736,0.356830,1.807175,-0.421026,-0.103245,0.586323,0.687753,-0.352590,-1.142716,-1.255676,...,0.898362,-0.293484,-0.292616,0.769751,0.698349,1.003053,-0.252433,-0.539774,0.279857,1.362519


In [36]:
import os
import joblib
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from lightgbm import LGBMRegressor
import xgboost as xgb
from sklearn.metrics import mean_absolute_error

from sklearn.feature_selection import RFE

working_directory = 'D:/machine_learning/DFS/nascar/ML'
os.chdir(working_directory)

# needs hyperparameters
def lgbm_mod(): return LGBMRegressor(random_state = 1, n_estimators = 1000, learning_rate = 0.01, n_jobs = -1)
def xgb_mod(): return xgb.XGBRegressor(random_state = 1)
def rf_mod(): return RandomForestRegressor(random_state = 1, n_jobs = -1, n_estimators = 200, max_depth = 5)
def sgd_mod(): return SGDRegressor(max_iter=100, tol=1e-3, random_state = 1) # won't work with feature importance
def svm_mod(): return SVR(C=1.0, epsilon=0.2) # won't work with feature importance
def neigh_mod(): return KNeighborsRegressor()
def dt_mod(): return DecisionTreeRegressor(random_state=0)

""" MODEL SELECTION """

model = lgbm_mod()

"""                 """

#print possible features
print('possible features:', X_train_scaled.columns.tolist(), '\n')

# Fit model, make predictions with all features
model.fit(X_train_scaled, Y_train)

preds_all = model.predict(X_test)

pdf = pred_df[['Status', 'FP']].copy()
pdf['Pred_FP_all'] = preds_all

# save the initial model to disk
filename = 'models/nascar_model_allfeats.pkl'
joblib.dump(model, filename)

# get top 50 features
dset = pd.DataFrame({'attr':X_train.columns.tolist(),'importance':model.feature_importances_}).sort_values(by='importance', ascending=False).reset_index(drop=True)
attr50 = dset['attr'][0:50].tolist()

# Using Top 50 Features, Find Top 30 Features
model.fit(X_train[attr50], Y_train)
dset = pd.DataFrame({'attr':X_train[attr50].columns.tolist(),'importance':model.feature_importances_}).sort_values(by='importance', ascending=False).reset_index(drop=True)
attr30 = dset['attr'][0:30].tolist()

# Using Top 30 Features, Find Top 20 Features
model.fit(X_train[attr30], Y_train)
dset = pd.DataFrame({'attr':X_train[attr30].columns.tolist(),'importance':model.feature_importances_}).sort_values(by='importance', ascending=False).reset_index(drop=True)
attr20 = dset['attr'][0:20].tolist()

#Perform RFE (recursive feature elimination) using Top 20 Features, To Find Top 15
rfe_model = RFE(model, n_features_to_select = 15)
rfe_model.fit(X_train[attr20], Y_train)
dset = pd.DataFrame({'attr':X_train[attr20].columns.tolist(),'importance':rfe_model.ranking_}).sort_values(by='importance', ascending=False).reset_index(drop=True)
cols15 = dset[dset['importance']==1]['attr'].tolist()

#Perform RFE using Top 15 Features, To Find Top 10
rfe_model = RFE(model, n_features_to_select = 10)
rfe_model.fit(X_train[cols15], Y_train)
dset = pd.DataFrame({'attr':X_train[cols15].columns.tolist(),'importance':rfe_model.ranking_}).sort_values(by='importance', ascending=False).reset_index(drop=True)
cols10 = dset[dset['importance']==1]['attr'].tolist()

print('T50 features', attr50, '\n')
print('T30 features', attr30, '\n')
print('T20 features', attr20, '\n')
print('T15 features',cols15, '\n')
print('T10 features',cols10, '\n')

model.fit(X_train[attr50], Y_train)
preds50 = model.predict(X_test[attr50])
filename = 'models/nascar_model_50feats.pkl'
joblib.dump(model, filename)

model.fit(X_train[attr30], Y_train)
preds30 = model.predict(X_test[attr30])
filename = 'models/nascar_model_30feats.pkl'
joblib.dump(model, filename)

model.fit(X_train[attr20], Y_train)
preds20 = model.predict(X_test[attr20])
filename = 'models/nascar_model_20feats.pkl'
joblib.dump(model, filename)

model.fit(X_train[cols15], Y_train)
preds15 = model.predict(X_test[cols15])
filename = 'models/nascar_model_15feats.pkl'
joblib.dump(model, filename)

model.fit(X_train[cols10], Y_train)
preds10 = model.predict(X_test[cols10])
filename = 'models/nascar_model_10feats.pkl'
joblib.dump(model, filename)


pdf['Pred_FP_50'] = preds50
pdf['Pred_FP_30'] = preds30
pdf['Pred_FP_20'] = preds20
pdf['Pred_FP_15'] = preds15
pdf['Pred_FP_10'] = preds10

pdf.to_csv('predictions_50_30_20_15_10.csv')

feature_sets = ['all', '50', '30', '20', '15', '10']

mae_values = [
    "{:.2f}".format(mean_absolute_error(Y_test, preds_all)),
    "{:.2f}".format(mean_absolute_error(Y_test, preds50)),
    "{:.2f}".format(mean_absolute_error(Y_test, preds30)),
    "{:.2f}".format(mean_absolute_error(Y_test, preds20)),
    "{:.2f}".format(mean_absolute_error(Y_test, preds15)),
    "{:.2f}".format(mean_absolute_error(Y_test, preds10))
]

results_df = pd.DataFrame({'Features' : feature_sets, 'MAE' : mae_values})

results_df.style.hide_index()
results_df.to_clipboard()
display(results_df)

possible features: ['LL_yr_cumsum', 'LC_yr_cumsum', 'DNF_yr_cumsum', 'Win_yr_cumsum', 'T5_yr_cumsum', 'T10_yr_cumsum', 'T20_yr_cumsum', 'FinPos_mean_L3', 'StrtPos_mean_L3', 'PlaceDiff_mean_L3', 'LL_sum_L3', 'LL_mean_L3', 'LC_sum_L3', 'LC_mean_L3', 'Pts_sum_L3', 'Pts_mean_L3', 'PPts_sum_L3', 'PPts_mean_L3', 'FP_sum_L3', 'FP_mean_L3', 'DNF_sum_L3', 'DNF_sum_L10', 'Win_sum_L3', 'T5_sum_L3', 'T10_sum_L3', 'T20_sum_L3', 'Win_sum_L10', 'T5_sum_L10', 'T10_sum_L10', 'T20_sum_L10', 'FinPos_mean_L3_track', 'StrtPos_mean_L3_track', 'PlaceDiff_mean_L3_track', 'LL_sum_L3_track', 'LL_mean_L3_track', 'LC_sum_L3_track', 'LC_mean_L3_track', 'Pts_sum_L3_track', 'Pts_mean_L3_track', 'PPts_sum_L3_track', 'PPts_mean_L3_track', 'FP_sum_L3_track', 'FP_mean_L3_track', 'DNF_sum_L3_track', 'Win_sum_L3_track', 'T5_sum_L3_track', 'T10_sum_L3_track', 'T20_sum_L3_track', 'FinPos_mean_L3_tt', 'StrtPos_mean_L3_tt', 'PlaceDiff_mean_L3_tt', 'LL_sum_L3_tt', 'LL_mean_L3_tt', 'LC_sum_L3_tt', 'LC_mean_L3_tt', 'Pts_sum_L3_t

,Features,MAE
0,all,27.94
1,50,12.17
2,30,12.34
3,20,12.31
4,15,12.95
5,10,13.05


In [381]:
pdf.loc[pdf['status'] == 'running'].describe()

,actual,predicted,abs. error
count,1797.000000,1797.000000,1797.000000
mean,53.519421,49.623383,11.473874
std,18.939117,14.068583,9.130235
min,0.000000,12.833677,0.002911
25%,39.700000,39.377014,4.421690
50%,53.400000,48.874165,9.301463
75%,66.000000,58.573917,16.168793
max,138.600000,108.717484,62.869211


In [382]:
pdf.loc[pdf['status'] != 'running'].describe()

,actual,predicted,abs. error
count,281.00000,281.000000,281.000000
mean,18.48363,41.552753,24.411830
std,14.62147,12.253642,15.295260
min,0.00000,13.525291,0.058690
25%,6.10000,32.670452,12.266153
50%,15.90000,39.381245,23.028477
75%,28.10000,48.775669,34.328568
max,73.60000,79.006569,70.062119


In [383]:
pdf.head(n=20)

,actual,predicted,abs. error,status
0,44.5,32.008884,12.491116,running
1,74.4,82.746796,8.346796,running
2,20.0,54.328568,34.328568,crash
3,77.0,57.456577,19.543423,running
4,72.7,52.579689,20.120311,running
5,56.8,56.631878,0.168122,running
6,37.2,38.757439,1.557439,running
7,44.9,35.598537,9.301463,running
8,40.0,39.028259,0.971741,running
9,39.7,34.268368,5.431632,electrical


In [384]:
pdf.describe()

,actual,predicted,abs. error
count,2078.000000,2078.000000,2078.000000
mean,48.781665,48.532024,13.223424
std,21.968089,14.107090,11.099790
min,0.000000,12.833677,0.002911
25%,33.900000,38.127890,4.790457
50%,49.800000,47.544008,10.416541
75%,63.575000,57.595137,18.772615
max,138.600000,108.717484,70.062119


---
**need to develop predictors based on track type**

* assign track type for each race, since it's known
* for each driver and track type, calculate means/sums for L3 of that track type